In [12]:
import csv
import pandas as pd
import math
import matplotlib.pyplot as plt
import re
import numpy as np
from numpy import *
from collections import Counter
from scipy.stats import shapiro
import warnings

from scipy import odr, stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LinearRegression
from lifelines import CoxPHFitter
warnings.filterwarnings('ignore')

In [15]:
# Read CSV
csv_file = 'QUBOClusteringData.csv'

originDf = pd.read_csv(csv_file)

originDf.head()

,tsolve,alg,nvars,seed,runseed
0,8.0,UEDA,5,1,1
1,7.0,GR,5,1,1
2,13.0,RS,5,1,1
3,73.0,UEDA,5,1,2
4,4.0,GR,5,1,2


Data Preparation

In [16]:
# Encoding algorithms in numerical values

encoder = LabelEncoder()
originDf['alg'] = encoder.fit_transform(originDf['alg'])

# Creating "event" based on the existence of a solution/time
originDf['event'] = ~originDf['tsolve'].isna()


In [17]:
def replace_nan_values(df, alg):
    df['seed'] = df['seed'].astype(int) 
    maxTSolverPerSeedUEDA = df[df["alg"] == alg].groupby("seed")['tsolve'].max().to_dict()

    for index, row in df.iterrows():
        if pd.isna(row["tsolve"]):
            seed_value = row["seed"]
            if pd.isna(maxTSolverPerSeedUEDA.get(seed_value)):
                maxRuntimeOverall = df['tsolve'].max()
            else:
                maxRuntimeOverall = maxTSolverPerSeedUEDA[seed_value]
            
            df.loc[index, "tsolve"] = maxRuntimeOverall


replace_nan_values(originDf, 'UEDA')
replace_nan_values(originDf, 'RS')
replace_nan_values(originDf, 'GR')

originDf


In [21]:
cph = CoxPHFitter()
cph.fit(originDf, duration_col='tsolve', event_col='event')

cph.print_summary()


<lifelines.CoxPHFitter: fitted with 7800 total observations, 2396 right-censored observations>
             duration col = 'tsolve'
                event col = 'event'
      baseline estimation = breslow
   number of observations = 7800
number of events observed = 5404
   partial log-likelihood = -44166.41
         time fit was run = 2023-12-27 15:31:06 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
alg        -0.81       0.45       0.02            -0.85            -0.76                 0.43                 0.47
nvars      -0.10       0.90       0.00            -0.11            -0.10                 0.90                 0.91
seed        0.01       1.01       0.00            -0.00             0.02                 1.00                 1.02
runseed    -0.00       1.00       0.00            -0.01             0.01                 0.99                 1.01

            cmp to      z      p   -log2(p)
covariate                                  
alg           0.00 -35.92 <0.005     936.32
nvars         0.00 -45.92 <0.005        inf
seed          0.00   1.85   0.06       3.95
runseed       0.00  -0.09   0.93       0.10
---
Concordance = 0.76
Partial AIC = 88340.83
log-likelihood ratio test = 3376.45 on 4 df
-log2(p) of ll-ratio test = inf